In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.4 MB/s eta 0:00:00


In [ ]:
! pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [ ]:
! pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 16.7 MB/s eta 0:00:00


In [1]:
import copy
import logging
from dataclasses import dataclass, field
import json
from json import loads, dumps
#import xlsxwriter
import pandas as pd
from tqdm.auto import tqdm
#import sentencepiece
import torch
import torchvision
from torch.utils.data import Dataset
from typing import Optional, Dict, Sequence
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Часть 1. Перевод инструкций

### Способ 1. "Huggingface"

(первый способ, который переводил > 1000 инструкций)

In [ ]:
# Файлик от alpaca_data.json
data_alpaca = json.load(open('alpaca_data.json'))

In [ ]:
# Модель для перевода
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
model.to('cuda')

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62518, 512, padding_idx=62517)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62518, 512, padding_idx=62517)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [ ]:
def to_ru(text):
    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True).to('cuda'))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [ ]:
to_ru(data_alpaca[4055]['output'])

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Это была прекрасная ночь, и звёзды сияли в темном небе. Я вышел наружу и посмотрел на них в благоговении. Я почувствовал весь свой стресс и беспокойство, когда я пил в красоте ночного неба. Я почувствовал чувство покоя и довольства, которое приходит только с глядя на звёзды. Это был идеальный конец идеального дня. Я улыбнулся и поблагодарил Вселенную за такое прекрасное зрелище. Я держал дыхание, воспользовавшись моментом и помня, как я был счастлив видеть такой чудесный момент.'

Несколько строчек для постепенного перевода и дозаписывания в файл `alpaca_data_ru.json`.

In [ ]:
start = 1

In [ ]:
all_ru = []
for instr_dict in tqdm(data_alpaca[start:]):
    instruction_ru = to_ru(instr_dict['instruction'])
    input_ru = to_ru(instr_dict['input'])
    output_ru = to_ru(instr_dict['output'])
    dict_ru = {'instruction': instruction_ru,
               'input': input_ru,
               'output': output_ru}
    all_ru.append(dict_ru)

  0%|          | 0/45776 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
start += len(all_ru)
start

797

In [ ]:
with open('alpaca_data_ru.json', 'a') as outf:
    outf.write(json.dumps(all_ru, indent=4, ensure_ascii=False))

В файл дозаписывается список словарей. Потом руками надо стереть границу между получившимися в файле списками.

Этим способом за раз получается перевести около 4000 инструкций. А потом надо ждать, пока снова заработает (примерно день).

### Способ 2. "Привет, гуглпереводчик"

In [ ]:
!wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json


In [ ]:
data_alpaca = json.load(open('alpaca_data.json'))

In [ ]:
instructions = []
inputs = []
outputs = []
for elem in tqdm(data_alpaca[:]):
    instructions.append(elem['instruction'])
    inputs.append(elem['input'])
    outputs.append(elem['output'])

100%|██████████| 52002/52002 [00:00<00:00, 1138764.90it/s]


In [ ]:
len(instructions), len(inputs), len(outputs)

(52002, 52002, 52002)

In [ ]:
# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('data_eng_52002.xlsx')
worksheet = workbook.add_worksheet()
wrap = workbook.add_format({'text_wrap': True})

# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

# Iterate over the data and write it out row by row.
for instruction, input, output in zip(instructions[40000:],
                                      inputs[40000:],
                                      outputs[40000:]):
    worksheet.write(row, col,     instruction)
    worksheet.write(row, col + 1, input)
    worksheet.write(row, col + 2, output)
    row += 1

workbook.close()

А теперь по частям переводим таблички (https://translate.google.com/)

И соединяем в json.

In [ ]:
table1 = pd.read_excel('data_10000_ru.xlsx', header=None,
                      names=['instruction', 'input', 'output'])
table2 = pd.read_excel('data_20000_ru.xlsx', header=None,
                      names=['instruction', 'input', 'output'])
table3 = pd.read_excel('data_30000_ru.xlsx', header=None,
                      names=['instruction', 'input', 'output'])
table4 = pd.read_excel('data_40000_ru.xlsx', header=None,
                      names=['instruction', 'input', 'output'])
table5 = pd.read_excel('data_52002_ru.xlsx', header=None,
                      names=['instruction', 'input', 'output'])
frames = [table1, table2, table3, table4, table5]
result = pd.concat(frames)
result = result.fillna("")
result.shape

(52002, 3)

In [ ]:
rujson = result.to_json(orient="records")
parsed = loads(rujson)

with open('alpaca_alldata_ru.json', 'a') as outf:
    outf.write(json.dumps(parsed, indent=4, ensure_ascii=False))

Вот файл с переведёнными инструкциями: https://github.com/tbkazakova/compling_nlp_hse_course/blob/main/alpaca_alldata_ru.json

## Часть 2. Дообучение модели

In [2]:
! pip install transformers==4.27.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.1 MB/s eta 0:00:00


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
import transformers
from transformers import Trainer

In [5]:
model_name_or_path = "ai-forever/rugpt3small_based_on_gpt2"

https://github.com/ai-forever/ru-gpts

In [6]:
IGNORE_INDEX = -100
PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведена инструкция, описывающая задачу, в сочетании с входными данными, которые обеспечивают дополнительный контекст."
        "Напишите ответ, который надлежащим образом завершает запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Ниже приведена инструкция, описывающая задачу."
        "Напишите ответ, который надлежащим образом завершает запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [7]:
def _tokenize_fn(strings: Sequence[str],
                 tokenizer: GPT2Tokenizer) -> Dict:
    """Tokenize a list of strings."""
    
    # tokenized_list = [
    #     tokenizer(
    #         text,
    #         return_tensors="pt",
    #         padding="longest",
    #         max_length=tokenizer.model_max_length,
    #         truncation=True,
    #     )
    #     for text in strings
    # ]
    tokenized_list = []
    for i, text in enumerate(tqdm(strings)):
        try:
          tokenized = tokenizer(
            text,
            return_tensors="pt",
            # padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
          )
          tokenized_list.append(tokenized)

        except ValueError as e:
          print(e)
          print(i, type(text), text, len(text), sep="\n")
          raise e
          # pass


    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [8]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: GPT2Tokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [9]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: GPT2Tokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: GPT2Tokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [10]:
model = GPT2LMHeadModel.from_pretrained(model_name_or_path,
        max_length=512,
        cache_dir="huggingface_cache").cuda()

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained(
    model_name_or_path,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False
)

In [12]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens({'eos_token': '<|endoftext!|>'})
# со встроенным tokenizer.eos_token_id почему-то был равен None

1

In [13]:
train_dataset = SupervisedDataset(tokenizer=tokenizer,
                                  data_path="alpaca_alldata_ru.json")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

  0%|          | 0/52002 [00:00<?, ?it/s]

  0%|          | 0/52002 [00:00<?, ?it/s]

In [14]:
train_args = transformers.TrainingArguments(learning_rate=1e-5, 
                 num_train_epochs=1,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="mydir",
                 fp16=True
                 )

In [15]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset, 
                 eval_dataset=None,
                 data_collator=data_collator)

In [16]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,3.092100
2000,2.674200
3000,2.576500
4000,2.524900
5000,2.512600
6000,2.502100
7000,2.489800
8000,2.461800
9000,2.425600
10000,2.419900


TrainOutput(global_step=26001, training_loss=2.4293190664724404, metrics={'train_runtime': 2828.4921, 'train_samples_per_second': 18.385, 'train_steps_per_second': 9.193, 'total_flos': 4118488070400000.0, 'train_loss': 2.4293190664724404, 'epoch': 1.0})

In [30]:
trainer.train()

Step,Training Loss
1000,2.217900
2000,2.203500
3000,2.226500
4000,2.215500
5000,2.214600


Step,Training Loss
1000,2.217900
2000,2.203500
3000,2.226500
4000,2.215500
5000,2.214600
6000,2.218800
7000,2.217800
8000,2.186400
9000,2.204300
10000,2.202400


TrainOutput(global_step=26001, training_loss=2.215574102628039, metrics={'train_runtime': 2799.0122, 'train_samples_per_second': 18.579, 'train_steps_per_second': 9.289, 'total_flos': 4118129814528000.0, 'train_loss': 2.215574102628039, 'epoch': 1.0})

In [ ]:
trainer.save_model('rugpt3small_tuned')

## Часть 3. Примеры работы дообученной модели

In [17]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Ниже приведена инструкция, описывающая задачу, в сочетании с входными данными, которые обеспечивают дополнительный контекст."
              "Напишите ответ, который надлежащим образом завершает запрос.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt], 
                        return_tensors="pt", padding=True)
    inputs.to('cuda')

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
#         max_length=100,
        max_new_tokens=20,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     do_sample=True, 
    #     top_k=30, 
    #     top_p=0.8, 
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs["input_ids"][0]):], skip_special_tokens=True)
    return summaries[0]

In [19]:
instruction = "Решите уравнение, найдите x"
text = "2x - 4 = 0"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' х = 2. равно 3.. равно 4..'

In [31]:
# 2 эпохи
instruction = "Решите уравнение, найдите x"
text = "2x - 4 = 0"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' х = 2чь.построение х = 2.'

In [20]:
instruction = "Продолжите текст"
text = "Меня зовут "
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Меня зовут......тами.. Он ('

In [32]:
# 2 эпохи
instruction = "Продолжите текст"
text = "Меня зовут "
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Меня зовут [Ваше имя]......'

In [21]:
instruction = "Резюмируйте этот текст"
text = """Иван Топорышкин пошел на охоту,
С ним пудель пошел, перепрыгнув забор,
Иван, как бревно провалился в болото,
А пудель в реке утонул, как топор."""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Иван Топорышкин, который прыгнул в реку, прыгнул в болото, как топор, как топор'

In [35]:
# 2 эпохи
instruction = "Резюмируйте этот текст"
text = """Иван Топорышкин пошел на охоту,
С ним пудель пошел, перепрыгнув забор,
Иван, как бревно провалился в болото,
А пудель в реке утонул, как топор."""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Иван Топорышкин, прыгнув в канаву, провалился в болото, как бревно.'

In [22]:
instruction = "Продолжите стихотворение"
text = "Муха села на варенье"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Муха села на варенье., В поисках варенья..'

In [47]:
# 2 эпохи
instruction = "Продолжите стихотворение"
text = "Муха села на варенье"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Муха села на варенье. Молниятахствиемствием'

In [29]:
# с переводом не очень, но это понятно, в датасете же всё перевелось на русский
instruction = "Переведите с английского на русский"
text = "Hello"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Привет, как дела? (CRMe) (GP'

In [49]:
# 2 эпохи
# с переводом не очень, но это понятно, в датасете же всё перевелось на русский
instruction = "Переведите с английского на русский"
text = "Hello"
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Привет, я так рада, что ты здесь!'

In [23]:
instruction = "А вы разговариваете?"
text = ""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Я разговариваю с вами...-1 штраф.'

In [50]:
# 2 эпохи
instruction = "А вы разговариваете?"
text = ""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Я разговариваю с друзьями и семьей. — это когда вы общаетесь с кем-то'

In [24]:
instruction = "А вы мыслите?"
text = ""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Я мыслю.......'

In [51]:
# 2 эпохи
instruction = "А вы мыслите?"
text = ""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Я думаю, что мысль о том, что мы можем быть свободны, может быть очень полезна для того'

In [25]:
instruction = "Какой сегодня день?"
text = ""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Сегодня пятница.: сегодня пятница.....'

In [61]:
# 2 эпохи
instruction = "Какой сегодня день?"
text = ""
predict_for_instruction(instruction, text, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Сегодня пятница.........'

Итого: очень задумчивая маленькая модель.